<a href="https://colab.research.google.com/github/Vanshikamahajan08/testproject1/blob/main/workout_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()


Saving fitness_exercises.csv to fitness_exercises.csv


{'fitness_exercises.csv': b'bodyPart,equipment,gifUrl,id,name,target\nwaist,body weight,http://d205bpvrqc9yn1.cloudfront.net/0001.gif,0001,3/4 sit-up,abs\nwaist,body weight,http://d205bpvrqc9yn1.cloudfront.net/0002.gif,0002,45\xc2\xb0 side bend,abs\nwaist,body weight,http://d205bpvrqc9yn1.cloudfront.net/0003.gif,0003,air bike,abs\nupper legs,body weight,http://d205bpvrqc9yn1.cloudfront.net/1512.gif,1512,all fours squad stretch,quads\nwaist,body weight,http://d205bpvrqc9yn1.cloudfront.net/0006.gif,0006,alternate heel touchers,abs\nback,cable,http://d205bpvrqc9yn1.cloudfront.net/0007.gif,0007,alternate lateral pulldown,lats\nlower legs,body weight,http://d205bpvrqc9yn1.cloudfront.net/1368.gif,1368,ankle circles,calves\nback,body weight,http://d205bpvrqc9yn1.cloudfront.net/3293.gif,3293,archer pull up,lats\nchest,body weight,http://d205bpvrqc9yn1.cloudfront.net/3294.gif,3294,archer push up,pectorals\nwaist,body weight,http://d205bpvrqc9yn1.cloudfront.net/2355.gif,2355,arm slingers hanging

In [2]:
import pandas as pd
df = pd.read_csv('/content/fitness_exercises.csv')
print(df.head())



     bodyPart    equipment                                         gifUrl  \
0       waist  body weight  http://d205bpvrqc9yn1.cloudfront.net/0001.gif   
1       waist  body weight  http://d205bpvrqc9yn1.cloudfront.net/0002.gif   
2       waist  body weight  http://d205bpvrqc9yn1.cloudfront.net/0003.gif   
3  upper legs  body weight  http://d205bpvrqc9yn1.cloudfront.net/1512.gif   
4       waist  body weight  http://d205bpvrqc9yn1.cloudfront.net/0006.gif   

     id                     name target  
0     1               3/4 sit-up    abs  
1     2            45° side bend    abs  
2     3                 air bike    abs  
3  1512  all fours squad stretch  quads  
4     6  alternate heel touchers    abs  


In [4]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab') # Download the necessary resource for sentence tokenization
from nltk.tokenize import word_tokenize

# Tokenize exercise names
df['tokens'] = df['name'].apply(word_tokenize)
print("Tokenized Example:", df['tokens'][0])  # e.g., ['Band', 'Pull', '-', 'Apart']

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Tokenized Example: ['3/4', 'sit-up']


In [5]:
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
df['cleaned_text'] = df['name'].apply(
    lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words])
)
print("After Stopword Removal:", df['cleaned_text'][0])  # e.g., "Band Pull-Apart"

After Stopword Removal: 3/4 sit-up


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
df['lemmatized'] = df['cleaned_text'].apply(
    lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()])
)
print("Lemmatized Example:", df['lemmatized'][0])

[nltk_data] Downloading package wordnet to /root/nltk_data...


Lemmatized Example: 3/4 sit-up


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=50)
tfidf_matrix = tfidf.fit_transform(df['lemmatized'])
keywords = tfidf.get_feature_names_out()
print("Top 10 Keywords:", keywords[:10])

Top 10 Keywords: ['arm' 'back' 'ball' 'band' 'bar' 'barbell' 'bench' 'bent' 'biceps'
 'cable']


In [8]:
def process_input(user_query):
    # Tokenize and clean
    tokens = word_tokenize(user_query.lower())
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Detect body part
    body_parts = ['chest', 'back', 'legs', 'arms', 'core', 'abs', 'shoulders']
    for word in tokens:
        if word in body_parts:
            return word
    return None  # If no body part found

In [9]:
def generate_workout(body_part):
    exercises = df[df['bodyPart'].str.lower() == body_part.lower()]
    if exercises.empty:
        return pd.DataFrame({"Message": [f"No exercises found for {body_part}. Try: legs/back/chest"]})
    return exercises.sample(3)[['name', 'equipment', 'target']]

# Test
user_input = "home chest workout"
body_part = process_input(user_input)
if body_part:
    print(generate_workout(body_part))
else:
    print("Please specify a body part (e.g., 'chest', 'legs')")

                                                name    equipment     target
862                          isometric chest squeeze  body weight  pectorals
432                  chest dip (on dip-pull-up cage)  body weight  pectorals
536  dumbbell incline one arm press on exercise ball     dumbbell  pectorals


In [11]:
!pip install gradio
import gradio as gr
import pandas as pd

def workout_generator(user_input):
    body_part = process_input(user_input)
    if not body_part:
        return pd.DataFrame({"Error": ["❌ Specify a body part (e.g., 'chest workout')"]})
    return generate_workout(body_part)

gr.Interface(
    fn=workout_generator,
    inputs="text",
    outputs="dataframe",
    examples=["15 min leg workout", "home chest exercise", "upper body routine"],
    title="💪 NLP Workout Generator",
    description="Describe your workout (e.g., '20 min home abs workout')"
).launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 114.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.0 MB/s eta 0:00:00
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0cc285e03b8e7b7a56.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory